In [2]:
from alphagenome.data import genome
from alphagenome.models import dna_client

/opt/conda/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at alphagenome/protos/dna_model.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at alphagenome/protos/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [4]:
# Use a tiny window to keep it fast
seq_len = dna_client.SUPPORTED_SEQUENCE_LENGTHS["SEQUENCE_LENGTH_2KB"]

# Any valid variant/interval will do; we just need the object to query tracks.
# (Use your current variant/interval if you already have them.)
dummy_var = genome.Variant(chromosome="chr1", position=1_000_000, reference_bases="A", alternate_bases="G")
dummy_int = dummy_var.reference_interval.resize(seq_len)

# Request just 1–2 modalities to minimize compute; CAGE + RNA_SEQ are good for breadth.
output = dna_model.predict_variant(
    interval=dummy_int,
    variant=dummy_var,
    organism=organism,                     # your enum from earlier
    requested_outputs=[
        dna_client.OutputType.CAGE,
        dna_client.OutputType.RNA_SEQ,
    ],
    ontology_terms=None,                   # IMPORTANT: discover across
)

NameError: name 'dna_model' is not defined

In [ ]:
import pandas as pd

def _safe_meta(t):
    if t is None or getattr(t, "values", None) is None or t.values.shape[-1] == 0:
        return pd.DataFrame(columns=["output_type","biosample_name","ontology_curie","biosample_type","strand","name"])
    m = t.metadata.copy()
    # ensure common columns exist
    for col in ["biosample_name","ontology_curie","biosample_type","strand","name"]:
        if col not in m.columns:
            m[col] = pd.NA
    return m